In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# Install virtualenv first (in Kaggle base python)
!pip install --quiet virtualenv

# Create env
!virtualenv /kaggle/working/agri_env

# Activate it
!source /kaggle/working/agri_env/bin/activate


In [ ]:
# --- HARD RESET packaging in Kaggle ---
import sys, os, importlib.util, shutil

!pip install -q --upgrade pip setuptools wheel

# Pinned versions that avoid MRO conflict
!pip install "pydantic==1.10.*" "typing-extensions==4.12.2" "langchain==0.2.11" "langchain-community==0.2.10"

# Remove all packaging traces from site-packages
for p in sys.path:
    if p.endswith("site-packages") and os.path.exists(p):
        for item in os.listdir(p):
            if item.startswith("packaging"):
                try:
                    target = os.path.join(p, item)
                    if os.path.isdir(target):
                        shutil.rmtree(target)
                    else:
                        os.remove(target)
                except Exception as e:
                    print("skip remove", target, e)

# Force reinstall clean version
!pip install --no-cache-dir --force-reinstall packaging==20.0

# Make sure Python picks the new one first
site_packages = "/kaggle/working/agri_env/lib/python3.11/site-packages"
if site_packages not in sys.path:
    sys.path.insert(0, site_packages)

# Reload packaging
import packaging
print("✅ Packaging fixed:", packaging.__version__, "from", packaging.__file__)

# Test Transformers now
from transformers import pipeline
print("✅ Transformers import works")


In [ ]:
!pip install --upgrade pip setuptools wheel
!pip install torch==2.3.1 torchvision==0.18.1 torchaudio==2.3.1 \
  --extra-index-url https://download.pytorch.org/whl/cu121
!pip install "transformers>=4.46.1,<4.47.0" sentence-transformers==3.2.1 accelerate==1.1.1 langchain-core
!pip install langchain==0.3.7 langchain-community==0.3.7 chromadb==0.5.5
!pip install unstructured==0.16.5 pdfminer.six==20240706 pillow==10.4.0 python-magic==0.4.27
!pip install pandas==2.2.2 numpy==1.26.4 requests==2.32.3 tqdm httpx==0.28.1
!pip install openai-whisper==20230314 parler-tts==0.2.2
!pip install ipykernel "scipy==1.11.4" 
!pip install --force-reinstall --no-deps packaging==25.0
!python -m ipykernel install --user --name=agri_env --display-name "Python (agri_env)"
!pip install --quiet \
  numba==0.61.0 \
  tenacity==8.2.3 \
  pydantic==2.11.4 \
  google-api-core>=2.19 \
  "google-cloud-bigquery-storage<3.0.0,>=2.30.0" \
  fsspec==2025.3.2
  

In [ ]:
!pip install --force-reinstall --no-cache-dir \
  torch==2.3.1+cu121 torchvision==0.18.1+cu121 rich==12.4.4 torchaudio==2.3.1+cu121 rich==12.4.4 \
  --extra-index-url https://download.pytorch.org/whl/cu121


In [ ]:
# --- CLEAN EVERYTHING CONFLICTING ---
!pip uninstall -y pydantic pydantic-core langchain langchain-core langchain-community fastapi datamodel-code-generator

# --- INSTALL COMPATIBLE VERSIONS (pre-Pydantic v2) ---
!pip install "pydantic==1.10.8"
!pip install "langchain==0.0.352"
!pip install "langchain-core==0.1.4"
!pip install "langchain-community==0.0.21"

# --- REINSTALL REST OF STACK ---
!pip install transformers==4.39.3 sentence-transformers==2.6.1
!pip install librosa==0.10.1 soundfile==0.12.1 opencv-python==4.9.0.80
!pip install git+https://github.com/huggingface/parler-tts.git
!pip install whisper-openai whisper
!pip install python-dotenv requests tqdm


In [ ]:
# 1) Purge every conflicting install
!pip uninstall -y pydantic pydantic-core langchain langchain-core langchain-community fastapi

# 3) Reinstall everything with constraints
!pip install "langchain<=0.0.352" \
            "langchain-core<=0.0.15" \
            "langchain-community<=0.0.17" \
            "transformers==4.39.3" \
            "sentence-transformers==2.6.1" 
            


In [ ]:
!pip uninstall -y torch torchaudio torchvision
!pip install torch==2.3.1 torchvision==0.18.1 torchaudio==2.3.1 
!pip install "transformers==4.46.1" "sentence-transformers==3.2.1" accelerate==1.1.1
!pip install "langchain==0.3.7" "langchain-community==0.3.7" "chromadb==0.5.5"
!pip install openai-whisper==20230314 parler-tts==0.2.2
!pip install spacy==3.7.5 nltk==3.9.1 gensim==4.3.3
!pip install librosa==0.10.2 soundfile==0.12.1 pydub==0.25.1 opencv-python==4.10.0.84
!pip install requests==2.32.3 httpx==0.28.1 tqdm==4.66.5 python-dotenv==1.0.1

# --- Imports ---
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Torch / Transformers
import torch
from transformers import pipeline, AutoTokenizer, AutoModel

# LangChain + Chroma (community versions!)
from langchain_community.vectorstores import Chroma
from langchain.schema import Document

# Whisper + TTS
import whisper
from parler_tts import ParlerTTSForConditionalGeneration

# NLP
import spacy, nltk, gensim

# Audio / CV (avoid torchaudio dependency issues)
import librosa, soundfile as sf
from pydub import AudioSegment
import cv2

# Utils
import requests, httpx, tqdm, os, sys, platform
from dotenv import load_dotenv

# --- Debug: show runtime versions ---
print("CUDA available:", torch.cuda.is_available(),
      "| Device:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else None)
print("torch:", torch.__version__,
      "| transformers:", __import__('transformers').__version__,
      "| langchain:", __import__('langchain').__version__)
print("Python:", sys.version,
      "| OS:", platform.platform())


In [ ]:
# --- Imports ---
!pip uninstall -y ydata-profiling whisper torch torchaudio torchvision
!pip install --quiet tenacity==8.2.3 langchain-openai==0.3.27 langchain_community langchain_core audiotoolbox pydub audio_utils
!pip install git+https://github.com/huggingface/parler-tts.git
!pip install torch==2.0.1 torchaudio==2.0.1 --extra-index-url https://download.pytorch.org/whl/cu117

# Core
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Torch / Transformers
import torch
from transformers import pipeline, AutoTokenizer, AutoModel

# LangChain + Chroma
from langchain import hub
from langchain_community.vectorstores import Chroma
from langchain.schema import Document

# Whisper + TTS
import whisper
from parler_tts import ParlerTTSForConditionalGeneration
from transformers import AutoTokenizer

# NLP
import spacy, nltk, gensim

# Audio / CV
import librosa, soundfile as sf, cv2

# Google / BigQuery
from google.cloud import bigquery

# Utils
import requests, httpx, tqdm, os, sys, platform
from dotenv import load_dotenv


# --- Debug: show runtime versions ---
print("CUDA available:", torch.cuda.is_available(), 
      "| Device:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else None)
print("torch:", torch.__version__, 
      "| Python:", sys.version, 
      "| OS:", platform.platform())


In [ ]:
!pip install whisper

In [ ]:
from pathlib import Path
BASE = Path("/kaggle/working/agri_intellect")
for p in [BASE/"data", BASE/"chroma_db", BASE/"cache", BASE/"outputs"]:
    p.mkdir(parents=True, exist_ok=True)

# (Optional) copy any small uploaded files from /kaggle/input into working space
# Example: copy PDFs into our local data folder so unstructured can write sidecar files
import shutil, os
IN_ROOT = Path("/kaggle/input/agri-intellect")
for ds in IN_ROOT.iterdir():
    # pull common doc files to our data dir (skip big image datasets)
    for ext in (".pdf", ".csv", ".xlsx", ".xls"):
        for f in ds.rglob(f"*{ext}"):
            dest = BASE/"data"/f.name
            if not dest.exists():
                shutil.copy2(f, dest)


In [ ]:
import os
from kaggle_secrets import UserSecretsClient

user_secrets = UserSecretsClient()

# Fetch secrets safely
os.environ["OPEN_AI_API_KEY"] = user_secrets.get_secret("OPEN_AI_API_KEY")
os.environ["WEATHER_API"] = user_secrets.get_secret("WEATHER_API")
os.environ["HUGGING_FACE"] = user_secrets.get_secret("HUGGING_FACE")
os.environ["SERPAPI_KEY"] = user_secrets.get_secret("SERPAPI_KEY")
print("✅ Keys loaded from Kaggle Secrets:")
for k in ["OPEN_AI_API_KEY", "WEATHER_API", "HUGGING_FACE", "SERPAPI_KEY"]:
    print(k, "=", "✔️ set" if os.getenv(k) else "❌ missing")


In [ ]:
# 🔹 Clean uninstall of unstructured-related packages
!pip uninstall -y unstructured unstructured-inference pdfminer pdfminer.six pdfplumber

# 🔹 Install necessary replacements
!pip install pypdf==5.1.0 pymupdf==1.24.10
!pip install langchain langchain-community chromadb
!pip install sentence-transformers openpyxl xlrd

from pathlib import Path
import pandas as pd
from langchain.schema import Document
from langchain_community.document_loaders import CSVLoader, PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import HuggingFaceEmbeddings

# Paths
DATA_DIR = Path("/kaggle/input/agri-intellect/agri_intellect")
DB_DIR   = Path("/kaggle/working/agri_intellect/chroma_db")

# 1) Load PDFs
pdf_docs = []
for f in DATA_DIR.rglob("*.pdf"):
    pdf_docs.extend(PyPDFLoader(str(f)).load())

# 2) Load CSVs
csv_docs = []
for f in DATA_DIR.rglob("*.csv"):
    csv_docs.extend(CSVLoader(str(f)).load())

# 3) Load Excel with pandas instead of UnstructuredExcelLoader
excel_docs = []
for f in list(DATA_DIR.rglob("*.xlsx")) + list(DATA_DIR.rglob("*.xls")):
    df = pd.read_excel(f)
    text = df.to_string()  # convert entire sheet to text
    excel_docs.append(
        Document(page_content=text, metadata={"source": str(f)})
    )

# Combine all docs
all_docs = pdf_docs + csv_docs + excel_docs

# 4) Split into chunks
splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=150)
splits = splitter.split_documents(all_docs)

# 5) Embeddings + Chroma persistent store
emb = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
vstore = Chroma.from_documents(splits, embedding=emb, persist_directory=str(DB_DIR))
vstore.persist()

print("✅ Documents processed and stored in ChromaDB")
print("Total chunks:", len(splits))


In [ ]:
# ===========================
# Agri-Intellect — Full Integrated
# (Multimodal RAG + VLM auto-switch CLIP <-> CDDM )
# ===========================

import os, sys, json, time, math, sqlite3, traceback, re
from pathlib import Path
from typing import Optional, List, Dict, Tuple
import datetime

import requests
import numpy as np
import pandas as pd
from PIL import Image

# ML imports (assumes previously installed/pinned versions)
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from sentence_transformers import SentenceTransformer, util

# LangChain + Chroma
from langchain_community.vectorstores import Chroma
try:
    from langchain_huggingface import HuggingFaceEmbeddings
except Exception:
    from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.docstore.document import Document

# -----------------------
# CONFIG & PATHS
# -----------------------
DATA_DIR = Path("/kaggle/input/agri-intellect/agri_intellect")
PEST_DIR = DATA_DIR / "Pest_Dataset"
CHROMA_DIR = Path("/kaggle/working/agri_intellect/chroma_db")
PEST_EMB_CACHE = Path("/kaggle/working/pest_emb_dualmode.npz")
REMINDER_DB = Path("/kaggle/working/agri_reminders.db")
PESTICIDES_CSV = DATA_DIR / "Pesticides.csv"

# Environment keys (set in Kaggle Secrets)
WEATHER_API = os.environ.get("WEATHER_API", "")
SARVAM_API_KEY = os.environ.get("SARVAM_API_KEY", "")
SARVAM_BASE = os.environ.get("SARVAM_BASE", "https://api.sarvam.ai")
SERPAPI_KEY = os.environ.get("SERPAPI_KEY", "")
USE_CDDM_FLAG = os.environ.get("USE_CDDM", "0")  # "1" to prefer CDDM when possible
CDDM_IMPORT_NAME = os.environ.get("CDDM_IMPORT", "cddm")  # adapt if your CDDM lib import differs

# Models
EMB_MODEL_TEXT = "sentence-transformers/all-MiniLM-L6-v2"   # text embeddings for Chroma
CLIP_MODEL = os.environ.get("CLIP_MODEL", "clip-ViT-B-32")  # fallback clip
GEN_MODEL = os.environ.get("GEN_MODEL", "google/flan-t5-small")  # generator for RAG
WHISPER_MODEL = os.environ.get("WHISPER_MODEL", "small")

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
ONLINE = True
try:
    requests.get("https://www.google.com", timeout=2)
except Exception:
    ONLINE = False

print("Device:", DEVICE, "| ONLINE:", ONLINE, "| USE_CDDM_FLAG:", USE_CDDM_FLAG)

# -----------------------
# SAFE UTILITIES
# -----------------------
def load_table(path: Path) -> pd.DataFrame:
    if not path or not path.exists(): return pd.DataFrame()
    try:
        return pd.read_csv(path)
    except Exception:
        try:
            return pd.read_excel(path)
        except Exception:
            return pd.DataFrame()

# -----------------------
# RAG: load retriever
# -----------------------
def load_retriever(chroma_dir: Path = CHROMA_DIR, emb_model: str = EMB_MODEL_TEXT):
    if not chroma_dir.exists():
        raise FileNotFoundError(f"Chroma DB not found at {chroma_dir}. Run parsing to produce it.")
    emb = HuggingFaceEmbeddings(model_name=emb_model)
    v = Chroma(persist_directory=str(chroma_dir), embedding_function=emb)
    return v, v.as_retriever(search_kwargs={"k":4})

# -----------------------
# GENERATOR (flan-t5)
# -----------------------
_GEN_TOKENIZER = None
_GEN_MODEL = None
def load_generator(model_name: str = GEN_MODEL):
    global _GEN_TOKENIZER, _GEN_MODEL
    if _GEN_TOKENIZER is None or _GEN_MODEL is None:
        _GEN_TOKENIZER = AutoTokenizer.from_pretrained(model_name)
        _GEN_MODEL = AutoModelForSeq2SeqLM.from_pretrained(model_name)
        if DEVICE == "cuda":
            _GEN_MODEL = _GEN_MODEL.to("cuda")
    return _GEN_MODEL, _GEN_TOKENIZER

def generate_from_context(question_en: str, docs: List[Document], max_new_tokens: int=150) -> str:
    model, tok = load_generator()
    ctx = "\n\n".join([d.page_content for d in docs])
    prompt = (
        "You are Agri-Intellect. Use ONLY the context below to answer. If not in KB respond: 'I don’t know from the knowledge base.'\n\n"
        f"Context:\n{ctx}\n\nQuestion: {question_en}\n\nAnswer:"
    )
    inputs = tok(prompt, return_tensors="pt", truncation=True, max_length=1024)
    if DEVICE == "cuda":
        inputs = {k: v.to("cuda") for k,v in inputs.items()}
    out = model.generate(**inputs, max_new_tokens=max_new_tokens, num_beams=2, early_stopping=True)
    return tok.decode(out[0], skip_special_tokens=True).strip()

# -----------------------
# Sarvam / ASR / TTS / Translation
# -----------------------
def sarvam_translate(text: str, target_lang: str = "en") -> str:
    if not SARVAM_API_KEY or not ONLINE: return text
    try:
        r = requests.post(f"{SARVAM_BASE.rstrip('/')}/translate", json={"text": text, "target_lang": target_lang},
                          headers={"Authorization": f"Bearer {SARVAM_API_KEY}"}, timeout=12)
        r.raise_for_status()
        j = r.json()
        return j.get("translation") or j.get("translated_text") or j.get("text") or text
    except Exception:
        return text

def sarvam_asr(audio_path: str) -> str:
    if not SARVAM_API_KEY or not ONLINE: return ""
    try:
        with open(audio_path,"rb") as f:
            r = requests.post(f"{SARVAM_BASE.rstrip('/')}/asr", files={"file": f},
                              headers={"Authorization": f"Bearer {SARVAM_API_KEY}"}, timeout=60)
        r.raise_for_status()
        return r.json().get("text") or r.json().get("transcript","") or ""
    except Exception:
        return ""

def sarvam_tts(text: str, lang: str="hi", out_path: str="/kaggle/working/sarvam_tts.wav") -> Optional[str]:
    if not SARVAM_API_KEY or not ONLINE: return None
    try:
        r = requests.post(f"{SARVAM_BASE.rstrip('/')}/tts", json={"text": text, "lang": lang},
                          headers={"Authorization": f"Bearer {SARVAM_API_KEY}"}, timeout=60)
        r.raise_for_status()
        if r.headers.get("content-type","").startswith("audio"):
            with open(out_path,"wb") as f: f.write(r.content); return out_path
        j = r.json(); b64 = j.get("audio_base64") or j.get("audio")
        if b64:
            import base64
            with open(out_path,"wb") as f: f.write(base64.b64decode(b64)); return out_path
    except Exception:
        return None

# offline Whisper / faster-whisper fallback
def local_asr_whisper(audio_path: str) -> str:
    try:
        from faster_whisper import WhisperModel
        model = WhisperModel(WHISPER_MODEL, device=DEVICE)
        segments, _ = model.transcribe(audio_path)
        return " ".join(s.text.strip() for s in segments)
    except Exception:
        try:
            import whisper
            m = whisper.load_model(WHISPER_MODEL)
            r = m.transcribe(audio_path)
            return r.get("text","").strip()
        except Exception:
            return ""

def local_tts(text: str, out_path: str="/kaggle/working/tts_local.wav") -> Optional[str]:
    try:
        import pyttsx3
        engine = pyttsx3.init()
        engine.save_to_file(text, out_path)
        engine.runAndWait()
        return out_path
    except Exception:
        return None

def asr_transcribe(audio_path: str) -> str:
    if SARVAM_API_KEY and ONLINE:
        t = sarvam_asr(audio_path)
        if t: return t
    return local_asr_whisper(audio_path)

def tts_speak(text: str, lang: str="hi") -> Optional[str]:
    if SARVAM_API_KEY and ONLINE:
        p = sarvam_tts(text, lang)
        if p: return p
    return local_tts(text)

def translate_to_en(text: str) -> str:
    if SARVAM_API_KEY and ONLINE:
        return sarvam_translate(text, target_lang="en")
    return text

def translate_from_en(text: str, tgt: str="hi") -> str:
    if tgt and tgt != "en" and SARVAM_API_KEY and ONLINE:
        return sarvam_translate(text, target_lang=tgt)
    return text

# -----------------------
# Intent detection (SBERT templates)
# -----------------------
INTENT_TEMPLATES = {
    "pest_image": ["identify pest", "what is this insect", "disease on leaves"],
    "pest_text": ["what pesticide", "my leaves have", "insect on plant"],
    "crop_choice": ["which crop to sow", "what crop is suitable", "what to plant"],
    "weather": ["weather", "is it going to rain", "forecast"],
    "msp": ["msp", "minimum support price", "government price"],
    "irrigation": ["irrigate", "when to water", "how often to water"],
    "fertilizer": ["fertilizer", "fertiliser", "how much fertilizer"],
    "harvest": ["when to harvest", "harvest time"],
    "storage": ["store", "storage"],
    "livestock": ["livestock", "feeding", "vaccination"],
    "schemes": ["scheme", "government scheme", "pm kisan"],
    "reminder": ["remind me", "schedule", "set reminder"],
    "general_rag": ["how to", "what is", "explain", "tell me about"]
}
_INTENT_MODEL = SentenceTransformer("all-MiniLM-L6-v2")
_intent_embeddings = {k: _INTENT_MODEL.encode(v, convert_to_tensor=True, normalize_embeddings=True) for k,v in INTENT_TEMPLATES.items()}

def detect_intent(text: str) -> Tuple[str, float]:
    if not text: return "general_rag", 0.0
    q = _INTENT_MODEL.encode(text, convert_to_tensor=True, normalize_embeddings=True)
    best, best_score = None, -1.0
    for k, emb in _intent_embeddings.items():
        s = float(util.cos_sim(q, emb).max().item())
        if s > best_score:
            best_score = s; best = k
    return best, best_score

# -----------------------
# VLMAdapter: CLIP <-> CDDM auto-switch
# -----------------------
def _import_cddm_module():
    if USE_CDDM_FLAG != "1": return None
    try:
        mod = __import__(CDDM_IMPORT_NAME)
        print("Imported CDDM module:", CDDM_IMPORT_NAME)
        return mod
    except Exception:
        return None

_CDDM_MOD = _import_cddm_module()
_CDDM_AVAILABLE = (_CDDM_MOD is not None) and (DEVICE == "cuda")
print("CDDM available:", bool(_CDDM_AVAILABLE))

class VLMAdapter:
    def __init__(self, clip_model_name: str = CLIP_MODEL):
        self.mode = "clip"
        self.clip = None
        self.cddm = None
        self.text_model = SentenceTransformer("all-MiniLM-L6-v2")
        self._init(clip_model_name)

    def _init(self, clip_model_name):
        # try initialize CDDM placeholder
        if _CDDM_AVAILABLE:
            try:
                # Replace the following with your CDDM model load if local.
                # e.g., self.cddm = CDDM.load(...); self.mode='cddm'
                # If your CDDM is server-side, set self.cddm = None and provide API calls in embed_image
                self.cddm = getattr(_CDDM_MOD, "CDDMModel", None) or None
                self.mode = "cddm"
                print("VLMAdapter: attempting CDDM mode.")
            except Exception as e:
                print("CDDM init failed:", e)
                self.cddm = None
                self.mode = "clip"
        else:
            self.mode = "clip"
        # CLIP fallback
        try:
            self.clip = SentenceTransformer(clip_model_name)
            print("VLMAdapter: CLIP ready:", clip_model_name)
        except Exception as e:
            print("VLMAdapter: CLIP init error:", e)
            self.clip = None

    def set_mode(self, prefer_cddm: bool):
        if prefer_cddm and _CDDM_AVAILABLE:
            self.mode = "cddm"
        else:
            self.mode = "clip"
        print("VLMAdapter mode set to:", self.mode)

    def embed_image(self, image_path: str) -> Optional[np.ndarray]:
        # CDDM path (replace with your CDDM embedding interface)
        if self.mode == "cddm" and self.cddm:
            try:
                # Example: if your local CDDM has embed_image:
                if hasattr(self.cddm, "embed_image"):
                    vec = self.cddm.embed_image(image_path)
                    return np.asarray(vec, dtype=np.float32)
                # If CDDM uses server API, implement call here (POST image -> vector)
                # FALLBACK to CLIP if not implemented
                raise NotImplementedError("CDDM embed not implemented; falling back to CLIP")
            except Exception as e:
                print("CDDM embed failed:", e)
                return self._embed_with_clip(image_path)
        # CLIP fallback
        return self._embed_with_clip(image_path)

    def _embed_with_clip(self, image_path: str) -> Optional[np.ndarray]:
        if not self.clip:
            raise RuntimeError("No CLIP available")
        try:
            im = Image.open(image_path).convert("RGB")
            emb = self.clip.encode(im, convert_to_tensor=False, normalize_embeddings=True)
            return np.asarray(emb, dtype=np.float32)
        except Exception as e:
            print("CLIP embed error:", e); return None

    def embed_text(self, text: str) -> Optional[np.ndarray]:
        # If CDDM supports joint text-image embeddings, swap here
        if self.mode == "cddm" and self.cddm and hasattr(self.cddm, "embed_text"):
            try:
                vec = self.cddm.embed_text(text)
                return np.asarray(vec, dtype=np.float32)
            except Exception as e:
                print("CDDM embed_text failed:", e)
        # fallback SBERT
        try:
            emb = self.text_model.encode(text, convert_to_tensor=False, normalize_embeddings=True)
            return np.asarray(emb, dtype=np.float32)
        except Exception as e:
            print("Text embed error:", e); return None

VLM = VLMAdapter()

# -----------------------
# PestIndex: cache embeddings for pest dataset (uses VLM)
# -----------------------
def build_or_load_pest_index(pest_root: Path = PEST_DIR, cache_path: Path = PEST_EMB_CACHE):
    if cache_path.exists():
        try:
            data = np.load(cache_path, allow_pickle=True)
            idx = {k: [(p, np.array(e)) for p,e in data[k]] for k in data.files}
            print("Loaded pest index from", cache_path)
            return idx
        except Exception:
            print("Failed to load pest cache; rebuilding.")
    index = {}
    if not pest_root.exists(): return index
    for pest_dir in sorted([p for p in pest_root.iterdir() if p.is_dir()]):
        arr = []
        for img in pest_dir.glob("*.*"):
            if img.suffix.lower() not in [".jpg",".jpeg",".png","bmp","webp"]: continue
            try:
                vec = VLM.embed_image(str(img))
                if vec is None: continue
                arr.append((str(img), vec.tolist()))
            except Exception:
                continue
        if arr:
            index[pest_dir.name] = arr
    try:
        np.savez(cache_path, **index)
        print("Saved pest index to", cache_path)
    except Exception:
        pass
    return index

PEST_INDEX = build_or_load_pest_index()

def identify_pest(query_image_path: str, per_class_limit: int=6) -> Optional[Tuple[str,float]]:
    if not Path(query_image_path).exists(): return None
    qvec = VLM.embed_image(query_image_path)
    if qvec is None: return None
    best = (None, -1.0)
    for pest, refs in PEST_INDEX.items():
        scores = []
        for ppath, remb in refs[:per_class_limit]:
            try:
                s = float(util.cos_sim(torch.tensor(qvec), torch.tensor(np.array(remb))).item())
                scores.append(s)
            except Exception:
                continue
        if scores:
            avg = float(np.mean(scores))
            if avg > best[1]:
                best = (pest, avg)
    if best[0] is None: return None
    return best

# -----------------------
# Pesticide / MSP loaders
# -----------------------
PESTICIDES_DF = load_table(PESTICIDES_CSV)
def lookup_pesticide_for(pest_name: str) -> Dict:
    if PESTICIDES_DF.empty:
        return {"note":"Pesticides.csv missing"}
    df = PESTICIDES_DF
    matches = df[df.apply(lambda row: row.astype(str).str.lower().str.contains(pest_name.lower()).any(), axis=1)]
    if matches.empty:
        col0 = df.columns[0]
        matches = df[df[col0].astype(str).str.lower().str.contains(pest_name.lower())]
    if matches.empty:
        return {"note":"No entry found"}
    row = matches.iloc[0].to_dict()
    return {k:str(v) for k,v in row.items()}

def find_msp_table() -> Optional[Path]:
    for p in DATA_DIR.rglob("*msp*.csv"):
        return p
    return None

MSP_PATH = find_msp_table()
MSP_DF = load_table(MSP_PATH) if MSP_PATH else pd.DataFrame()
def lookup_msp(crop: str, yield_qty: Optional[float]=None) -> Dict:
    if MSP_DF.empty:
        return {"note":"MSP table missing"}
    cropcol = next((c for c in MSP_DF.columns if "crop" in c.lower() or "commodity" in c.lower()), MSP_DF.columns[0])
    pricecol = next((c for c in MSP_DF.columns if "msp" in c.lower() or "price" in c.lower()), None)
    if not pricecol:
        pricecol = MSP_DF.columns[1] if len(MSP_DF.columns)>1 else MSP_DF.columns[0]
    sel = MSP_DF[MSP_DF[cropcol].astype(str).str.lower() == crop.lower()]
    if sel.empty:
        sel = MSP_DF[MSP_DF[cropcol].astype(str).str.lower().str.contains(crop.lower())]
    if sel.empty:
        return {"note":"MSP not found"}
    try:
        price = float(str(sel.iloc[0][pricecol]).replace(",","").strip())
    except Exception:
        price = sel.iloc[0][pricecol]
    out = {"crop":crop, "msp":price}
    if yield_qty is not None:
        try: out["expected_revenue"] = float(price)*float(yield_qty)
        except Exception: out["expected_revenue"] = None
    return out

# -----------------------
# Agronomy heuristics (extendable)
# -----------------------
CROP_RULES = {
    "wheat": {"irrigation_days":10, "water_mm":35, "fertilizer":"N:120,P:60,K:40 kg/ha", "harvest_days":120, "storage":"Dry to 12% moisture"},
    "rice":  {"irrigation_days":5,  "water_mm":40, "fertilizer":"N:100,P:60,K:40 kg/ha", "harvest_days":135, "storage":"Dry to 14% moisture"},
    "maize": {"irrigation_days":7,  "water_mm":35, "fertilizer":"N:120,P:60,K:40 kg/ha", "harvest_days":100, "storage":"Dry to 13% moisture"},
}

def crop_agronomy(crop: str) -> Dict:
    k = crop.strip().lower()
    return CROP_RULES.get(k, {"note":"No heuristic available."})

# -----------------------
# Weather (indianapi.in) simple wrapper
# -----------------------
def fetch_weather(city: str) -> Dict:
    if not WEATHER_API or not ONLINE:
        return {"offline":True, "note":"Weather API not available"}
    try:
        r = requests.get("https://indianapi.in/weather-api", params={"city":city, "key":WEATHER_API}, timeout=10)
        r.raise_for_status()
        return r.json()
    except Exception as e:
        return {"error": str(e)}

def weather_recommendations(wx: Dict) -> List[Dict]:
    out = []
    for crop, meta in CROP_RULES.items():
        out.append({"crop":crop, **meta})
    return out

# -----------------------
# Reminders (SQLite)
# -----------------------
def reminders_init():
    conn = sqlite3.connect(str(REMINDER_DB))
    cur = conn.cursor()
    cur.execute("""CREATE TABLE IF NOT EXISTS reminders (id INTEGER PRIMARY KEY, task TEXT, next_due TEXT, interval_days INTEGER, meta TEXT)""")
    conn.commit(); conn.close()

def schedule_reminder(task: str, start_in_days:int=0, interval_days:int=1, meta:Optional[Dict]=None):
    reminders_init()
    next_due = (datetime.datetime.now() + datetime.timedelta(days=int(start_in_days))).isoformat()
    conn = sqlite3.connect(str(REMINDER_DB)); cur = conn.cursor()
    cur.execute("INSERT INTO reminders (task,next_due,interval_days,meta) VALUES (?,?,?,?)", (task, next_due, int(interval_days), json.dumps(meta or {})))
    conn.commit(); conn.close()
    return {"task":task, "next_due": next_due}

def check_reminders(now:Optional[datetime.datetime]=None):
    reminders_init()
    now = now or datetime.datetime.now()
    conn = sqlite3.connect(str(REMINDER_DB)); cur = conn.cursor()
    cur.execute("SELECT id,task,next_due,interval_days,meta FROM reminders")
    rows = cur.fetchall()
    due = []
    for r in rows:
        rid, task, next_due_s, interval, meta = r
        nd = datetime.datetime.fromisoformat(next_due_s)
        if nd <= now:
            due.append({"id":rid, "task":task, "meta": json.loads(meta)})
            new_due = (now + datetime.timedelta(days=interval)).isoformat()
            cur.execute("UPDATE reminders SET next_due=? WHERE id=?", (new_due, rid))
    conn.commit(); conn.close()
    return due

# -----------------------
# SERPAPI fallback augmentation
# -----------------------
def serpapi_snippets(query: str, num=2) -> Optional[str]:
    if not SERPAPI_KEY or not ONLINE: return None
    try:
        r = requests.get("https://serpapi.com/search", params={"q": query, "api_key": SERPAPI_KEY}, timeout=8)
        r.raise_for_status()
        js = r.json()
        items = js.get("organic_results") or []
        snippets = []
        for it in items[:num]:
            s = it.get("snippet") or it.get("title")
            if s: snippets.append(s)
        if snippets: return " ".join(snippets)
    except Exception:
        return None

# -----------------------
# HIGH-LEVEL USER ROUTER: handle_user_input
# -----------------------
def ensure_retriever():
    try:
        v, retr = load_retriever(CHROMA_DIR)
        return v, retr
    except Exception:
        return None, None

def handle_user_input(
    text: Optional[str]=None,
    audio_path: Optional[str]=None,
    image_path: Optional[str]=None,
    location: Optional[str]=None,
    lang_hint: Optional[str]=None,
    do_tts: bool=False
) -> Dict:
    """
    Single entry. Accepts text OR audio OR image.
    Returns dict: {intent, answer_en, answer_local, meta...}
    """
    user_lang = lang_hint or "en"
    user_text = ""
    if audio_path:
        user_text = asr_transcribe(audio_path)
    elif text:
        user_text = str(text)
    # PRIORITY: image diagnosis
    if image_path:
        # try multimodal chroma search first
        try:
            v, retr = ensure_retriever()
            emb = VLM.embed_image(image_path)
            docs = v.similarity_search_by_vector(emb.tolist(), k=4) if (v and emb is not None) else []
        except Exception:
            docs = []
        pest = None
        if docs:
            pc = docs[0].page_content
            if pc.lower().startswith("pest image of"):
                pest = pc.split("pest image of")[-1].strip()
        if not pest:
            pid = identify_pest(image_path)
            if pid:
                pest, conf = pid
        if pest:
            pestic = lookup_pesticide_for(pest)
            agr = crop_agronomy(pest) if pest.lower() in CROP_RULES else {}
            ans_en = f"Detected pest: {pest}. Pesticide info: {pestic}. Agronomy: {agr}"
        else:
            ans_en = "Could not identify pest from the image. Please upload a clearer photo or provide more details."
        ans_local = translate_from_en(ans_en, user_lang)
        if do_tts: tts_speak(ans_local, lang=user_lang)
        return {"intent":"pest_image", "answer_en": ans_en, "answer_local": ans_local, "pest": pest}

    # If no image: handle text-based intents
    if not user_text:
        return {"error":"No input provided"}

    # translate to English for internal processing
    text_en = translate_to_en(user_text)
    intent, score = detect_intent(text_en)
    if score < 0.35: intent = "general_rag"

    # RAG retriever
    try:
        v, retriever = load_retriever(CHROMA_DIR)
    except Exception:
        v, retriever = None, None

    # ROUTING
    if intent == "pest_text":
        # try extraction of pest phrase by checking folder names
        pest_candidates = [p.name for p in PEST_DIR.iterdir() if p.is_dir()] if PEST_DIR.exists() else []
        found = next((pc for pc in pest_candidates if pc.lower() in text_en.lower()), None)
        if found:
            pestic = lookup_pesticide_for(found)
            agr = crop_agronomy(found) if found.lower() in CROP_RULES else {}
            ans_en = f"Pest: {found}. Pesticide: {pestic}. Agronomy: {agr}"
        else:
            ans_en = "I couldn't find pest name in your message. Please upload an image for accurate diagnosis."
        ans_local = translate_from_en(ans_en, user_lang)
        if do_tts: tts_speak(ans_local, lang=user_lang)
        return {"intent":"pest_text", "answer_en": ans_en, "answer_local": ans_local}

    if intent in ("crop_choice","weather"):
        if location:
            wx = fetch_weather(location)
            recs = weather_recommendations(wx)
            lines = [f"{r['crop'].title()}: irrigate every {r.get('irrigation_days','?')} days; fertilizer: {r.get('fertilizer','see dataset')}" for r in recs]
            ans_en = "Weather: {}\nRecommendations:\n{}".format(("offline" if wx.get("offline") else "live"), "\n".join(lines))
        else:
            ans_en = "Please provide your location so I can fetch local weather and recommend crops."
        ans_local = translate_from_en(ans_en, user_lang)
        if do_tts: tts_speak(ans_local, lang=user_lang)
        return {"intent":"crop_choice", "answer_en":ans_en, "answer_local":ans_local}

    if intent == "msp":
        # extract crop name heuristically
        words = text_en.lower().split()
        candidates = list(CROP_RULES.keys()) + (MSP_DF.iloc[:,0].astype(str).str.lower().tolist() if not MSP_DF.empty else [])
        crop = next((c for c in candidates if c.lower() in text_en.lower()), None)
        if not crop:
            crop = words[-1]
        yield_qty = None
        for tok in words:
            try:
                if tok.replace(".","",1).isdigit():
                    yield_qty = float(tok); break
            except: pass
        res = lookup_msp(crop, yield_qty)
        if "note" in res:
            ans_en = res["note"]
        else:
            ans_en = f"MSP for {crop} is {res['msp']}. " + (f"Expected revenue: {res.get('expected_revenue')}" if res.get('expected_revenue') else "")
        ans_local = translate_from_en(ans_en, user_lang)
        if do_tts: tts_speak(ans_local, lang=user_lang)
        return {"intent":"msp", "answer_en":ans_en, "answer_local":ans_local, "msp":res}

    if intent in ("irrigation","fertilizer","harvest","storage","livestock"):
        found_crop = next((c for c in CROP_RULES.keys() if c.lower() in text_en.lower()), None)
        if found_crop:
            agr = crop_agronomy(found_crop)
            if intent == "irrigation":
                ans_en = f"For {found_crop.title()}: irrigate every {agr.get('irrigation_days')} days (~{agr.get('water_mm')} mm)."
            elif intent == "fertilizer":
                ans_en = f"For {found_crop.title()}: recommended fertilizer: {agr.get('fertilizer')}"
            elif intent == "harvest":
                ans_en = f"Expected harvest for {found_crop.title()} in ~{agr.get('harvest_days')} days from sowing."
            elif intent == "storage":
                ans_en = f"Storage: {agr.get('storage')}"
            else:
                ans_en = "Livestock: provide balanced feed, routine vaccination, clean housing, and monitor health."
        else:
            ans_en = "Please mention the crop or livestock type for specific guidance."
        ans_local = translate_from_en(ans_en, user_lang)
        if do_tts: tts_speak(ans_local, lang=user_lang)
        return {"intent":intent, "answer_en":ans_en, "answer_local":ans_local}

    if intent == "schemes":
        if retriever:
            docs = retriever.get_relevant_documents(text_en)
            if docs:
                ans_en = generate_from_context(text_en, docs)
            else:
                s = serpapi_snippets(text_en)
                ans_en = s or "No matching scheme info found in knowledge base."
        else:
            ans_en = "Knowledge base retriever not available."
        ans_local = translate_from_en(ans_en, user_lang)
        if do_tts: tts_speak(ans_local, lang=user_lang)
        return {"intent":"schemes", "answer_en":ans_en, "answer_local":ans_local}

    if intent == "reminder":
        num = re.findall(r"\d+", text_en)
        interval = int(num[0]) if num else 1
        task = text_en.split("to ",1)[1] if "to " in text_en else text_en
        sched = schedule_reminder(task.strip(), start_in_days=0, interval_days=interval)
        ans_en = f"Reminder scheduled: {task.strip()} every {interval} day(s)."
        ans_local = translate_from_en(ans_en, user_lang)
        return {"intent":"reminder", "answer_en":ans_en, "answer_local":ans_local, "schedule":sched}

    # fallback: general RAG
    if retriever:
        docs = retriever.get_relevant_documents(text_en)
        if docs:
            ans_en = generate_from_context(text_en, docs)
        else:
            s = serpapi_snippets(text_en)
            ans_en = s or "I don't know from the knowledge base."
    else:
        ans_en = "Knowledge base not available; please ensure Chroma DB exists."
    ans_local = translate_from_en(ans_en, user_lang)
    if do_tts: tts_speak(ans_local, lang=user_lang)
    return {"intent":"general_rag", "answer_en":ans_en, "answer_local":ans_local}

# -----------------------
# Public helper APIs
# -----------------------
def api_diagnose_pest(image_path: str, do_tts: bool=False) -> Dict:
    pid = identify_pest(image_path)
    if not pid:
        return {"detected": None, "message":"Could not identify pest."}
    pest, conf = pid
    pestic = lookup_pesticide_for(pest)
    agr = crop_agronomy(pest) if pest.lower() in CROP_RULES else {}
    msg = f"Detected {pest} (confidence {conf:.2f}). Pesticide info: {pestic}. Agronomy: {agr}"
    local = translate_from_en(msg, "hi")
    if do_tts: tts_speak(local, "hi")
    return {"detected":pest, "confidence":conf, "pesticide":pestic, "agronomy":agr, "message":msg}

def api_crop_planning_by_city(city: str, do_tts: bool=False) -> Dict:
    wx = fetch_weather(city)
    recs = weather_recommendations(wx)
    lines = [f"{r['crop'].title()}: irrigate every {r.get('irrigation_days','?')} days; fertilizer {r.get('fertilizer','see dataset')}" for r in recs]
    msg = "\n".join(lines)
    local = translate_from_en(msg, "hi")
    if do_tts: tts_speak(local, "hi")
    return {"weather":wx, "advice_en":msg, "advice_local":local}

def api_msp_quote(crop: str, yield_qty: Optional[float]=None, do_tts: bool=False) -> Dict:
    m = lookup_msp(crop, yield_qty)
    if "note" in m:
        local = translate_from_en(m["note"], "hi")
        return {"error":m["note"], "message_local":local}
    msg = f"MSP for {crop} is {m['msp']}." + (f" Expected revenue: {m.get('expected_revenue')}" if m.get("expected_revenue") else "")
    local = translate_from_en(msg, "hi")
    if do_tts: tts_speak(local, "hi")
    return {"msp":m, "message_local":local}

def api_schedule(task:str, start_in_days:int=0, interval_days:int=1, meta:Optional[Dict]=None):
    return schedule_reminder(task, start_in_days, interval_days, meta)

def api_check_reminders():
    return check_reminders()

def set_vlm_mode(prefer_cddm: bool):
    VLM.set_mode(prefer_cddm and _CDDM_AVAILABLE)
    global PEST_INDEX
    PEST_INDEX = build_or_load_pest_index()  # rebuild cache for consistency
    return {"mode": VLM.mode, "pest_classes": len(PEST_INDEX)}

# -----------------------
# Final: ensure multimodal Chroma upsert (non-destructive)
# -----------------------
def upsert_pest_images_into_chroma_if_needed(chroma_obj):
    # Upsert images as documents with embeddings — functionality depends on your Chroma client's API.
    try:
        count = 0
        for pest_folder in sorted([p for p in PEST_DIR.iterdir() if p.is_dir()]):
            for img in pest_folder.glob("*.*"):
                if img.suffix.lower() not in [".jpg",".jpeg",".png","bmp","webp"]: continue
                emb = VLM.embed_image(str(img))
                if emb is None: continue
                # Use chroma object's add_texts (client-specific) with embeddings if available
                try:
                    chroma_obj.add_texts(
                        texts=[f"Pest image of {pest_folder.name}"],
                        metadatas=[{"pest": pest_folder.name, "source": str(img), "type":"image"}],
                        embeddings=[emb.tolist()]
                    )
                    count += 1
                except Exception:
                    # some chroma clients don't accept direct embeddings; skip gracefully
                    pass
        if count>0:
            chroma_obj.persist()
        return count
    except Exception:
        return 0

# Print readiness
print("Agri-Intellect integrated module ready.")
print("Public functions: handle_user_input(), api_diagnose_pest(), api_crop_planning_by_city(), api_msp_quote(), api_schedule(), api_check_reminders(), set_vlm_mode(prefer_cddm)")
print("Current VLM mode:", VLM.mode)

# Example: set_vlm_mode(False)  # force CLIP
# Example: set_vlm_mode(True)   # prefer CDDM (if available)


In [ ]:
import gradio as gr
import tempfile
import os
from typing import Optional, Dict

# Assuming your entire Agri-Intellect code block is already defined above, including:
# - handle_user_input()
# - set_vlm_mode()
# - tts_speak(), etc.

def gradio_agri_intellect(
    text_input: str,
    audio_file: Optional[gr.Audio.Audio] = None,
    image_file: Optional[gr.Image.Image] = None,
    location: str = "",
    prefer_cddm: bool = False,
    enable_tts: bool = False,
    lang_hint: str = "en"
) -> str:

    # Save audio locally if provided
    audio_path = None
    if audio_file is not None:
        with tempfile.NamedTemporaryFile(delete=False, suffix=".wav") as f:
            f.write(audio_file.read())
            audio_path = f.name

    # Save image locally if provided
    image_path = None
    if image_file is not None:
        img_temp = tempfile.NamedTemporaryFile(delete=False, suffix=".png")
        image_file.save(img_temp.name)
        image_path = img_temp.name

    # Set VLM mode as per user toggle
    set_vlm_mode(prefer_cddm)

    # Run inference
    try:
        response = handle_user_input(
            text=text_input,
            audio_path=audio_path,
            image_path=image_path,
            location=location if location else None,
            lang_hint=lang_hint,
            do_tts=enable_tts
        )
    except Exception as e:
        response = {"error": f"Inference error: {str(e)}"}

    # Clean up temp files
    if audio_path and os.path.exists(audio_path):
        os.unlink(audio_path)
    if image_path and os.path.exists(image_path):
        os.unlink(image_path)

    # Compose output string
    output_lines = []

    if "error" in response:
        output_lines.append(f"❌ Error: {response['error']}")
    else:
        intent = response.get("intent", "unknown")
        output_lines.append(f"🧠 Detected Intent: {intent}\n")

        answer_en = response.get("answer_en") or response.get("message") or ""
        output_lines.append(f"🇬🇧 Answer (English):\n{answer_en}\n")

        answer_local = response.get("answer_local") or ""
        if answer_local and answer_local != answer_en:
            output_lines.append(f"🌐 Answer (Local Language - {lang_hint}):\n{answer_local}\n")

        # Optionally add extra info like pest name or MSP
        if "pest" in response:
            output_lines.append(f"🐞 Pest detected: {response['pest']}\n")
        if "msp" in response:
            output_lines.append(f"💰 MSP Data: {response['msp']}\n")
        if "schedule" in response:
            sched = response["schedule"]
            output_lines.append(f"⏰ Reminder Scheduled: Task '{sched.get('task')}' next due {sched.get('next_due')}\n")

    return "\n".join(output_lines)


with gr.Blocks(title="Agri-Intellect: Multimodal Agricultural Assistant") as demo:
    gr.Markdown(
        """
        # Agri-Intellect
        Multilingual, multimodal agricultural advisory system with:
        - Text, Audio (speech), Image input
        - Pest detection & pesticide suggestion
        - Crop planning by weather and location
        - MSP pricing info & harvesting/fertilizer/irrigation advice
        - Reminder scheduling
        - Auto-switch VLM (CDDM/CLIP)
        - Multilingual ASR, TTS & translation support
        """
    )

    with gr.Row():
        with gr.Column():
            text_input = gr.Textbox(label="Enter your query (any language)", lines=4, placeholder="Ask anything about agriculture...")
            audio_input = gr.Audio(source="microphone", type="file", label="Or speak your query")
            image_input = gr.Image(type="pil", label="Upload crop or pest image (optional)")
            location_input = gr.Textbox(label="Your city/location (for weather-based advice)", placeholder="e.g., Pune")
            lang_hint = gr.Dropdown(
                label="Language of your query",
                choices=["en", "hi", "gu", "mr", "pa", "kn", "te", "ta", "bn"],
                value="en",
                interactive=True
            )
            prefer_cddm = gr.Checkbox(label="Prefer CDDM mode for pest detection (if available)", value=False)
            enable_tts = gr.Checkbox(label="Enable Text-to-Speech for responses", value=False)
            submit_btn = gr.Button("Get Agricultural Advice")

        with gr.Column():
            output_box = gr.Textbox(label="Agri-Intellect Response", lines=20, interactive=False)

    submit_btn.click(
        fn=gradio_agri_intellect,
        inputs=[text_input, audio_input, image_input, location_input, prefer_cddm, enable_tts, lang_hint],
        outputs=[output_box]
    )

demo.launch(share=True)
